In [1]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 20.5 MB/s eta 0:00:00


In [2]:
from pymongo import MongoClient

client = MongoClient('mongodb+srv://cameron:<DSCExercise>@dsc-exercise-cluster.75t6l.mongodb.net/?retryWrites=true&w=majority&appName=DSC-Exercise-Cluster')
db = client['DS-Exercise-Cluster']

# example squat & bench data reports under a specific user
workout_log = {
    "user_id": "cameron",
    "session": [
        {
            "exercise_name": "Squat",
            "weight":225,
            "reps": 6,
            "sets": 3,
            "muscle_group_targeted": "glutes, quads, hamstrings, adductor, hip flexors, calves",
            "pain_reported": True,
            "pain_location": "lower back"
        },
        {
            "exercise_name": "Bench Press",
            "weight": 150,
            "reps": 5,
            "sets": 4,
            "muscle_group_targeted": "pectorals, front delts, triceps",
            "pain_reported": False
        }
    ]
}

# insert data into cluster
db.workout_logs.insert_one(workout_log)


ConfigurationError: The DNS query name does not exist: _mongodb._tcp.dsc-exercise-cluster.75t6l.mongodb.net.

In [ ]:
# Retrieve logs where pain was reported
pain_logs = db.workout_logs.find({"session.pain_reported": True})

for log in pain_logs:
    print(log)


In [3]:
pip install transformers accelerate sentencepiece

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM

model_name = "decapoda-research/llama-7b-hf"  # You can specify the version here

tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name, device_map="auto")


In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb+srv://cameron:<DSCExercise>@dsc-exercise-cluster.75t6l.mongodb.net/?retryWrites=true&w=majority&appName=DSC-Exercise-Cluster')
db = client['DS-Exercise-Cluster']

# Retrieve logs where pain was reported
pain_logs = db.workout_logs.find({"session.pain_reported": True})

# Let's use the first pain log as an example for RAG
retriever_result = list(pain_logs)[0]


In [ ]:
prompt = f"User has reported pain during {retriever_result['session'][0]['exercise_name']} targeting {retriever_result['session'][0]['muscle_group_targeted']}. " \
         f"Suggest an alternative exercise and potential rehabilitation exercises."


In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(inputs.input_ids, max_length=100)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


In [ ]:
from fastapi import FastAPI
from transformers import LlamaTokenizer, LlamaForCausalLM
from pymongo import MongoClient

app = FastAPI()

# load in LLAMA
model_name = "decapoda-research/llama-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name, device_map="auto")

# MongoDB
client = MongoClient('mongodb+srv://cameron:<DSCExercise>@dsc-exercise-cluster.75t6l.mongodb.net/?retryWrites=true&w=majority&appName=DSC-Exercise-Cluster')
db = client['DS-Exercise-Cluster']

@app.get("/generate_coaching")
def generate_coaching(user_id: str):
    # check for any pain history under pain logs
    pain_logs = db.workout_logs.find_one({"user_id": user_id, "session.pain_reported": True})

    if not pain_logs:
        return {"error": "No pain history found for the user."}

    # prompt & generation via LLAMA
    prompt = f"User has reported pain during {pain_logs['session'][0]['exercise_name']} " \
             f"targeting {pain_logs['session'][0]['muscle_group_targeted']}. " \
             f"Suggest an alternative exercise and potential rehabilitation exercises."

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs.input_ids, max_length=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {"response": response}
